## Presentation and Discussion of Results

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2

#### Establishing a connection to the database
Database is hosted on local server. The database is called "Flight Delays "

In [ ]:
# Connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="Flight Delays ",
    user="postgres",
    password="12345678"
)

In [ ]:
# Create a cursor object
cur = conn.cursor()

#### Importing Queries
Queries are stored in a file called "Queries.sql". We import the queries and store them in a list.

In [ ]:
file_path = "Objective 3_Queries.sql"
with open(file_path, "r") as file:
    queries = file.read().split(';')

In [ ]:
queries = [query.strip() for query in queries]

#### Query 1: Setting negative delays to 0
To ensure our data is not skewed by negative delays, we set all negative delays to 0. To use this adjusted data we commit it to the database.

In [ ]:
# Select Flights with Delay only
cur.execute(queries[0])
conn.commit()

#### Query 2: Average Delay by Latitude
To explore the relationship between latitude and delay, we plot the delay for each latitude. We also calculate the correlation coefficient between latitude and delay.

In [ ]:
# Load query 1 results into a dataframe
cur.execute(queries[1])
results = cur.fetchall()
latitude_delays = pd.DataFrame(results, columns=['Latitude', 'Delay'])


In [ ]:
# Scatterplot of Latitude vs. Delay
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Latitude', y='Delay', data=latitude_delays)
plt.title('Arrival Delay vs. Latitude of Origin Airport')
plt.xlabel('Latitude of Origin Airport')
plt.ylabel('Arrival Delay (minutes)')
plt.show()

In [ ]:
# Calculate the correlation coefficient
latitude_delays = latitude_delays.dropna()
corr = np.corrcoef(latitude_delays['Latitude'], latitude_delays['Delay'])[0, 1]
print('Correlation coefficient:', corr)

#### Query 3: Average Delay
To further investigate the relationship, we group the latitudes into ranges of 5 degrees and calculate the average delay for each range. We then plot these averages against the latitude ranges and calculate the correlation coefficient.

In [ ]:
# Average Delay by Latitude
avg_delays = pd.DataFrame(columns=['Latitude_lower',
                                'Latitude_upper',
                                'Latitude Range',
                                'Average'])

In [ ]:
# Latitude ranges for grouping
latitude_ranges = [
    (0.0, 20.0),    # For WHERE A.Latitude < 20.0
    (20.0, 25.0),
    (25.0, 30.0),
    (30.0, 35.0),
    (35.0, 40.0),
    (40.0, 45.0),
    (45.0, 50.0),
    (50.0, 55.0),
    (55.0, 60.0),
    (60.0, 65.0),
    (65.0, 90.0)    # For WHERE A.Latitude > 65.0
]

In [ ]:
# Average delay for each latitude range
for i, bounds in enumerate(latitude_ranges):
    cur.execute(queries[1], bounds)
    results = cur.fetchall()
    avg_delays.loc[i] = [bounds[0], bounds[1], str(bounds), results[0][0]]
  

In [ ]:
# Plot the results
plt.figure(figsize=(15, 6))
sns.barplot(x='Latitude Range', y='Average', data=avg_delays)
plt.title('Average Arrival Delay by Latitude of Origin Airport')
plt.xlabel('Latitude of Origin Airport')
plt.ylabel('Average Arrival Delay (minutes)')
plt.show()

In [ ]:
# calculate the correlation coefficient
corr = np.corrcoef(avg_delays['Latitude_lower'], avg_delays['Average'])[0, 1]
print('Correlation coefficient:', corr)

#### Closing the connection and cursor
To prevent data integrity issues, we close the connection and cursor.

In [ ]:
# Close the cursor and connection
cur.close()
conn.close()